In [ ]:
import sys

import matplotlib.pyplot as plt

%matplotlib notebook
%matplotlib notebook
## save variables
import pickle
## folder names
from glob import glob
## standard libraries
import numpy as np

#!{sys.executable} -m pip install tensorflow-gpu --user
#!{sys.executable} -m pip install keras --user
import tensorflow as tf
from keras import backend as k

import keras
from keras.optimizers import SGD, Adam
#!{sys.executable} -m pip install hyperas --user
#!{sys.executable} -m pip install networkx==1.11 --user

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

import os.path
import datetime

from dependencies import models
from dependencies import functions

from functools import partial

import pandas as pd
import time

In [ ]:
%matplotlib notebook

In [ ]:
df = pd.DataFrame(columns=['name', 'acc', 'val_acc', 'val_top3_acc', 'test_acc',
                           'feature', 'delta', 'inference_time', 'augmented_set', 'parameters', 'epochSGD', 'epochAdam'])
line = {'name': None, 'acc':None, 'val_acc':None, 'val_top3_acc':None, 'test_acc':None,
                           'feature':None, 'delta':None, 'inference_time':None, 
                            'augmented_set':None, 'parameters':None, 'epochSGD': None, 'epochAdam' : None}

# Creating dataset

In [ ]:
#choose which dictionary to use
choice =    'mfcc'
useDelta =  True

data = {}
    
selected = 0

if choice == 'mfcc':
    #retrieving of used values for the computation of mfcc
    with open('variables/mfccValues.pkl', 'rb') as f:  
        values = pickle.load(f)
    for index in range(4):
        #name format of the selected data
        if useDelta:
            name = 'variables/mfccDictDD'+functions.getName(index)+'[nC='+str(values[selected][0])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
        else:
            name = 'variables/mfccDict'+functions.getName(index)+'[nC='+str(values[selected][0])+' wL='+str(values[selected][2])+' wS='+str(values[selected][3])+'].pkl'
        #loading in usedDict of the mfcc dict
        with open(name, 'rb') as f: 
            data[functions.getName(index)] = pickle.load(f)
        print('Loaded '+name)

elif choice == 'logfilter':
    #retrieving of used values for the computation of mfcc
    with open('variables/lfValues.pkl', 'rb') as f:  
        values = pickle.load(f)
    for index in range(4):
        #name format of the selected data
        if useDelta:
            name = 'variables/logfiltDictDD'+functions.getName(index)+'[nF='+str(values[selected][0])+' wL='+str(values[selected][1])+' wS='+str(values[selected][2])+'].pkl'
        else:
            name = 'variables/logfiltDict'+functions.getName(index)+'[nF='+str(values[selected][0])+' wL='+str(values[selected][1])+' wS='+str(values[selected][2])+'].pkl'
        #saving in usedDict of the logfilter dict
        with open(name, 'rb') as f:  
            data[functions.getName(index)] = pickle.load(f)
        print('Loaded '+name)

In [ ]:
#core words of the dataset
coreKey = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero",
           "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]

#split of the core set
numbers = ['one', 'two', 'three','four','five','six','seven','eight','nine', "zero"]

words = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

#selecting the subset of words to predict
usedLabels = words

usedLabels.append('silence')

unknownLabels = list(data['Train'].keys())
for key in usedLabels:
    try:
        unknownLabels.remove(key)
    except:
        print(key, ' not in used')

In [ ]:
%%time
#divding between train and test with also scaling data
functions.train_test_creator(
    data,
    usedLabels,
    unknownLabels,
    with_unknown = False,
    scalerType = 'robust',
    depth = (len(data['Train'][words[0]].shape)-3)*2 + 1,
    unknown_percentage = 0.3)

print()

# Import data

In [ ]:
dataset, labels = functions.load_dataset()
with open('variables/labelList.pkl', 'rb') as f: 
        labelList = pickle.load(f)

# Fitting

## Fitting variables

In [ ]:
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total 0.8 of the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.8
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

epoch = 20
epochSGD = 20

epochs = [epoch, epochSGD]

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001/epoch, amsgrad=True)
sgd = SGD(lr=0.001, decay=0.001/epochSGD, momentum=0.9, nesterov=True)

optimizers = [adam, sgd]

top3_acc = partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.name = 'top3_acc'

titles = ['Adam History', 'SGD History']

dest_directory = 'model_backup/'
if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)

esCallBack = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto', baseline=None)

# Sequential Models

## Model1

In [ ]:
%%time
name = 'model1'
print(name)
table = line
table['name'] =  name
table['augmented_set'] =  'No'

inputData, inputLabel, testData, testLabel, validData, validLabel, augmentedData, augmentedLabel, validation_data, loss_weights = functions.modelSelection(name, dataset, labels)

cnn = models.model1(inputData,inputLabel)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs

table['feature'] =  choice
table['delta'] =  useDelta
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(1):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
%%time
table['augmented_set'] =  'Yes'
cnn = models.model1(inputData,inputLabel)

print('Adding augmented dataset')
inputData, inputLabel = functions.meltData(inputData, augmentedData, inputLabel, augmentedLabel, 0.7)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=3,#epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

    
functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs
        
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

## Model2

In [ ]:
%%time
name = 'model2'
print(name)
table = line
table['name'] =  name
table['augmented_set'] =  'No'

inputData, inputLabel, testData, testLabel, validData, validLabel, augmentedData, augmentedLabel, validation_data, loss_weights = functions.modelSelection(name, dataset, labels)

cnn = models.model2(inputData,inputLabel, baseDim = 40)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))


functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs


table['feature'] =  choice
table['delta'] =  useDelta
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
%%time
table['augmented_set'] =  'Yes'
cnn = models.model2(inputData,inputLabel, baseDim = 40 )

print('Adding augmented dataset')
inputData, inputLabel = functions.meltData(inputData, augmentedData, inputLabel, augmentedLabel, 0.7)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs
        
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

## TinyDarknet

In [ ]:
%%time
name = 'tinyDarknet'
print(name)
table = line
table['name'] =  name
table['augmented_set'] =  'No'

inputData, inputLabel, testData, testLabel, validData, validLabel, augmentedData, augmentedLabel, validation_data, loss_weights = functions.modelSelection(name, dataset, labels)

cnn = models.tinyDarknet(inputData,inputLabel, dropout = 0.3)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs
        
table['feature'] =  choice
table['delta'] =  useDelta
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
%%time
table['augmented_set'] =  'Yes'
cnn = models.tinyDarknet(inputData,inputLabel, dropout = 0.3)

print('Adding augmented dataset')
inputData, inputLabel = functions.meltData(inputData, augmentedData, inputLabel, augmentedLabel, 0.7)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))
    
functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs
        
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

# Inception

## Single input

In [ ]:
%%time
name = 'SiSoInc'
print(name)
table = line
table['name'] =  name
table['augmented_set'] =  'No'
inputData, inputLabel, testData, testLabel, validData, validLabel, augmentedData, augmentedLabel, validation_data, loss_weights = functions.modelSelection(name, dataset, labels)

cnn = models.SiSoInception(inputData,inputLabel)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs

table['feature'] =  choice
table['delta'] =  useDelta
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
%%time
table['augmented_set'] =  'Yes'
cnn = models.SiSoInception(inputData,inputLabel, dropout = 0.5)

print('Adding augmented dataset')
inputData, inputLabel = functions.meltData(inputData, augmentedData, inputLabel, augmentedLabel, 0.7)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData.shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))
    
functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs

table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData.shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

## Multiple input

In [ ]:
%%time
name = 'MiSoInc'
print(name)
table = line
table['name'] =  name
table['augmented_set'] =  'No'
inputData, inputLabel, testData, testLabel, validData, validLabel, augmentedData, augmentedLabel, validation_data, loss_weights = functions.modelSelection(name, dataset, labels)

cnn = models.MiSoInception(inputData, inputLabel, dropout = 0.4)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData[0].shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))

functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs

table['feature'] =  choice
table['delta'] =  useDelta
table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData[0].shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
%%time
table['augmented_set'] =  'Yes'
cnn = models.MiSoInception(inputData,inputLabel, dropout = 0.4)

print('Adding augmented dataset')
inputData, inputLabel = functions.meltData(inputData, augmentedData, inputLabel, augmentedLabel, 0.7)

fittedHistory = []

for count, optimizer in enumerate(optimizers):
    print('Using optimizer number ' + str(count))
    cnn.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy', top3_acc ], loss_weights=loss_weights)
    fittedHistory.append(cnn.fit(inputData, inputLabel,
                         epochs=epochs[count],
                         batch_size=round(inputData[0].shape[0]/400),
                         shuffle=True,
                         validation_data=validation_data,
                         callbacks = [esCallBack]))
    
functions.plotHistory(epochs, fittedHistory, 'Training History')

for count, fitted in enumerate(fittedHistory):
    n_epochs = len(fitted.history['loss'])
    if count == 0:
        table['epochSGD'] =  n_epochs
    else:
        table['epochAdam'] =  n_epochs

table['parameters'] =  cnn.count_params()

for key in fittedHistory[-1].history:
    if key in df.keys():
        table[key] =  fittedHistory[-1].history[key][-1]
        
micros = int(round(time.time() * 1000000))
# 100 times the prediction of test data
for i in range(100):
    preds = cnn.predict(testData)
    
micros = time.time() * 1000000 - micros
# normalize the time for single prediction
micros = micros/100/testData[0].shape[0]

table['inference_time'] =  micros

#Plot normalized confusion matrix
functions.plot_confusion_matrix(preds, testLabel, classes=labelList, normalize=True,
                      title='Normalized confusion matrix')
precision = cnn.evaluate(testData,  testLabel)
print ("Precision: ", round(precision[1]*100,2),"%")

table['test_acc'] =  round(precision[1],4)


df = df.append(table, ignore_index=True)

In [ ]:
with open('variables/dataFrame.pkl', 'wb') as f:  
        pickle.dump(df, f)

In [ ]:
df